Importar las librerias que se necesitan

In [218]:
import pandas as pd
import os
import glob
import yfinance as yf
from datetime import datetime
import math
import numpy as np

Descargar los datos necesarios desde Yahoo Finance (se puede automatizar con airflow)

In [219]:
"""

## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos 
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock
for archivo_csv in archivos_csv:
    # Obtener el símbolo del stock desde el nombre del archivo
    nombre_archivo = os.path.basename(archivo_csv)
    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock
    
    # Descargar los datos desde Yahoo Finance para los últimos 10 años
    data = yf.download(simbolo, start='2013-01-01')
    
    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)
    data = data[['Open', 'Close', 'Adj Close', 'Volume']]
    
    # Guardar los datos en un nuevo archivo CSV
    nuevo_nombre_archivo = f'{simbolo}.csv'
    ruta_nuevo_archivo = os.path.join(directorio, nuevo_nombre_archivo)
    data.to_csv(ruta_nuevo_archivo)

"""

"\n\n## PARA DESCARGAR LA NUEVA INFORMACION - TARDÓ 120 Minutos, 87 Minutos \n# Ruta al directorio donde se encuentran los archivos CSV\ndirectorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'\n\n# Patrón para buscar archivos CSV\npatron = '*.csv'\n\n# Obtener la ruta de directorio de cada uno\narchivos_csv = glob.glob(os.path.join(directorio, patron))\n\n# Descargar los datos de Yahoo Finance y almacenarlos en un DataFrame por cada stock\nfor archivo_csv in archivos_csv:\n    # Obtener el símbolo del stock desde el nombre del archivo\n    nombre_archivo = os.path.basename(archivo_csv)\n    simbolo = nombre_archivo.split('.')[0]  # Suponemos que el nombre del archivo es el símbolo del stock\n    \n    # Descargar los datos desde Yahoo Finance para los últimos 10 años\n    data = yf.download(simbolo, start='2013-01-01')\n    \n    # Filtrar las columnas necesarias (Precio de apertura, cierre, cierre ajustado y volumen)\n    data = data[['Op

Se extraen los nombres de las acciones de NASDAQ y NYSE

In [242]:
# Ruta al directorio donde se encuentran los archivos CSV
directorio = 'C:/Users/Leidy Hernandez/Desktop/Soy Henry/Proyecto integrador/fh_20181104/full_history'

# Patrón para buscar archivos CSV
patron = '*.csv'

# Obtener la ruta de directorio de cada uno
archivos_csv = glob.glob(os.path.join(directorio, patron))

# Lista para almacenar los nombres de los archivos CSV
nombres_archivos_csv = []

# Extraer el nombre de cada archivo CSV
for archivo in archivos_csv:
    nombres_archivos_csv.append(os.path.basename(archivo))

Se crea un DataFrame por cada Stock

In [243]:
# Lista para almacenar los DataFrames de los archivos CSV
dataframes_csv = []
# Leer cada archivo CSV y almacenar el DataFrame correspondiente
for i in range(len(nombres_archivos_csv)):
    ruta_archivo = directorio + '/' + nombres_archivos_csv[i]  # Ruta completa al archivo
    df = pd.read_csv(ruta_archivo)
    dataframes_csv.append(df)

# Crea el DF por accion
for i in range(len(nombres_archivos_csv)):
    # Extraer un nombre de la lista
    nombre_variable = nombres_archivos_csv[i].replace('.csv','')
    # Asignar un valor a la variable con el nombre extraído
    globals()[nombre_variable] = dataframes_csv[i]

Se crea una lista con el nombre de cada stock (No la ruta del archivo)

In [244]:
# Extraer sólo el nombre del stock
for i in range(len(nombres_archivos_csv)):
    nombres_archivos_csv[i] = nombres_archivos_csv[i].replace('.csv','')

Se empiezan a eliminar los Stocks que no se pueden utilizar 

Se eliminan los df que no tengan datos, que no tengan histórico desde el primer día hábil del 2013, que no estén cotizando para el día hábil más cercano de cuando descargamos los datos, y que tengan información continua (que no falte ninguna cotización)

In [246]:
archivos_inutiles = []
length = len(globals()[nombres_archivos_csv[0]]['Date'])
ult_fecha = A['Date'][len(A['Date'])-1]

# Iterar sobre los nombres de archivos CSV
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    fecha = dataframe['Date']
    
    if len(fecha) != length or fecha[0] != '2013-01-02'  or fecha[len(fecha) - 1] != ult_fecha:
        archivos_inutiles.append(nombre_archivo)

# Eliminar los DataFrames inútiles y sus nombres de archivos CSV
for nombre_archivo in archivos_inutiles:
    nombre_dataframe = nombre_archivo.replace('.csv', '')
    del globals()[nombre_dataframe]
    nombres_archivos_csv.remove(nombre_archivo)

Calcular Desviación Estandard y rango de precios 

In [248]:
metrics = pd.DataFrame()

metrics['Stock'] = nombres_archivos_csv
std = []

for i in range(len(metrics['Stock'])):
    std.append(np.std(globals()[metrics['Stock'][i]]['Adj Close']))

metrics['Standard Deviation'] = std


In [249]:
r_min = []
r_max = []
fecha = '2023-07-14'    ## Fecha de 'Ayer', del dia anterior para basarse ena estimación del precio de "hoy" (Siempre tiene que ser de días hábiles)
indice = globals()[metrics['Stock'][0]].loc[A['Date'] == fecha].index[0]    ## Como todos los archivos tienen la misma cantidad de datos, si en uno se encuentra la fecha en x 
                                                                            ## posicion entonces en esa misma estará para todos los demás df
for i in range(len(metrics['Stock'])):
    r_min.append(globals()[metrics['Stock'][i]]['Close'][indice] - metrics['Standard Deviation'][i])
    r_max.append(globals()[metrics['Stock'][i]]['Close'][indice] + metrics['Standard Deviation'][i])

metrics['Rango Mínimo Estimado'] = r_min
metrics['Rango Máximo Estimado'] = r_max

Calcular rendimientos diarios, mensuales y anuales con su respectivo promedio

In [250]:
## PARA CALCULAR LOS RENDIMIENTOS DIARIOS -> (Pf/Pi)-1
for j in range(len(nombres_archivos_csv)):
    nombre_archivo = nombres_archivos_csv[j]
    dataframe = globals()[nombre_archivo]
    dataframe['Rendimientos diarios (%)'] = np.nan
    adj_close = dataframe['Adj Close']
    rendimientos = [((adj_close[i] / adj_close[i-1]) - 1)*100 for i in range(1, len(adj_close))]
    dataframe['Rendimientos diarios (%)'][1:] = rendimientos    

C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_12096\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_12096\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Rendimientos diarios (%)'][1:] = rendimientos
C:\Users\Leidy Hernandez\AppData\Local\Temp\ipykernel_12096\3329189856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [251]:
## CALCULAR PROMEDIO DE RENDIMIENTOS DIARIOS POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(globals()[metrics['Stock'][i]]['Rendimientos diarios (%)']))
metrics['Rendimiento promedio diario (%)'] = p_rend

In [252]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_mensuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]

    # Antes de aplicar resample, convertimos la columna 'Date' en un índice de fecha
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    monthly_data = dataframe.resample('M', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    monthly_data['Rendimiento mensual (%)'] = (monthly_data['Adj Close']['last'] / monthly_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_mensuales[nombre_archivo] = monthly_data

# Ahora se puede acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']


In [253]:
## CALCULAR PROMEDIO DE RENDIMIENTOS MENSUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_mensuales[metrics['Stock'][i]]['Rendimiento mensual (%)']))
metrics['Rendimiento promedio mensual (%)'] = p_rend

In [254]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK

# Crear un diccionario para almacenar los DataFrames con los rendimientos mensuales
rendimientos_anuales = {}

# Calcular el rendimiento mensual para cada DataFrame
for nombre_archivo in nombres_archivos_csv:
    dataframe = globals()[nombre_archivo]
    
    # Agrupar los datos por año y mes y obtener el primer y último valor de "Adj Close" de cada mes
    annual_data = dataframe.resample('A', on='Date').agg({'Adj Close': ['first', 'last']})
    
    # Calcular el rendimiento del mes
    annual_data['Rendimiento Anual (%)'] = (annual_data['Adj Close']['last'] / annual_data['Adj Close']['first'] - 1) * 100
    
    # Almacenar el DataFrame con los rendimientos mensuales en el diccionario
    rendimientos_anuales[nombre_archivo] = annual_data

# Ahora puedes acceder a los DataFrames individuales con sus nombres
# por ejemplo, dataframes_rendimientos_mensuales['NombreDelDataFrame']

In [255]:
## CALCULAR PROMEDIO DE RENDIMIENTOS ANUALES POR STOCK
p_rend = []
for i in range(len(metrics['Stock'])):
    p_rend.append(np.mean(rendimientos_anuales[metrics['Stock'][i]]['Rendimiento Anual (%)']))
metrics['Rendimiento promedio anual (%)'] = p_rend

Calcular volatilidad utilizando los métodos Historical Volatility (HV) y Realized Volatility (RV) 

In [256]:
## METODO "Historical Volatility (HV)" 
    # HV = √VAR * √N
    # VAR = SD/N

var = []
HV = []
dias_negociados = len(A['Date'])
for i in range(len(metrics['Stock'])):
    var.append((metrics['Standard Deviation'][i])/(dias_negociados))
    HV.append(math.sqrt(var[i]) * math.sqrt(dias_negociados))

In [257]:
## METODO "Realized Volatility (RV)" 
    # RV = √((∑SDR)/N)
    # SDR = (Daily return)^2
SDR = []  # Para agregar la suma de cada stock

for j in range(len(metrics['Stock'])):
    sdr = []  # Reiniciar la lista de SDR para cada stock
    for i in range(1, dias_negociados):
        sdr.append((globals()[metrics['Stock'][j]]['Rendimientos diarios (%)'][i]) ** 2)
    SDR.append(sum(sdr))

# Ahora calculamos la volatilidad (RV) para cada stock
RV = pd.DataFrame()
RV['Stock'] = metrics['Stock']
RV['Realized Volatility (RV)'] = [((sdr_sum / dias_negociados) ** 0.5) for sdr_sum in SDR]


    Baja volatilidad: Menos del 10% de desviación estándar.
    Volatilidad moderada: Entre el 10% y el 20% de desviación estándar.
    Alta volatilidad: Más del 20% de desviación estándar.

Calcular valor intrínseco 

    Primero hay que calcular:
        Terminal Value (TV) 
        Terminal Growth Rate (g)
        Retention Ratio ***

In [258]:
## WEB SCRAPING PARA EXTRAER EL NET INCOME

def get_net_income(ticket):
    url = f"https://www.marketwatch.com/investing/stock/{ticket}/financials/income/quarter"

    # Realizar una solicitud GET a la página web
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de respuesta 200)
    if response.status_code == 200:
        # Parsear el contenido HTML de la página con BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Buscar el elemento que contiene el net income
        net_income_element = soup.find("div", class_="cell__content", string="Net Income")

        if net_income_element:
            # Obtener todas las etiquetas div con clase "cell__content" dentro de la fila del net income
            net_income_values = net_income_element.find_parent("tr").find_all("div", class_="cell__content")

            # Acceder al penúltimo elemento de la lista para obtener el valor del net income
            net_income = net_income_values[-2].text.strip()
            return net_income
        else:
            return "Net Income no encontrado en la tabla de Income Statement."
    else:
        return "No se pudo acceder a la página web."



In [280]:
net_income = []
for i in range(len(metrics['Stock'])):
    net_income.append(get_net_income(metrics['Stock'][i]))
metrics['Net Income'] = net_income

In [304]:
metrics.head(-2080)

,Stock,Standard Deviation,Rango Mínimo Estimado,Rango Máximo Estimado,Rendimiento promedio diario (%),Rendimiento promedio mensual (%),Rendimiento promedio anual (%),Net Income
0,A,40.380049,79.369951,160.130049,0.070364,1.242421,17.123556,302M
1,AA,14.011931,20.508070,48.531931,0.067836,0.980131,16.992027,(102M)
2,AAL,12.521633,5.618366,30.661633,0.059130,0.903905,11.778864,1.34B
3,AAME,0.751532,1.068468,2.571532,0.065728,0.106849,-3.691327,(1.45M)
4,AAON,21.376307,78.073690,120.826304,0.114815,2.041958,30.095244,36.81M
...,...,...,...,...,...,...,...,...
799,EGO,11.618924,-0.528924,22.708924,0.004484,-1.046363,-1.363821,2.06M
800,EGP,47.229313,138.270687,232.729313,0.070885,1.390944,19.729394,43.45M
801,EGY,2.251801,1.718199,6.221801,0.071939,1.306668,3.995617,3.45M
802,EHC,14.520287,52.739715,81.780289,0.073355,1.433448,16.175341,88.1M


In [305]:
len(metrics['Stock'])-2080

804